In [7]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import hopsworks
import pandas as pd
import numpy as np

In [9]:
project = hopsworks.login(project="id2223_enric")
fs = project.get_feature_store("id2223_enric")

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197783
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
hackernews_df = pd.read_csv("./pd_combined.csv")
hackernews_df['id'] = hackernews_df['id'].astype(float).astype(np.int32)
hackernews_df['title'] = hackernews_df['title'].astype(str)
hackernews_df['url'] = hackernews_df['url'].astype(str)
hackernews_df['descendants'] = hackernews_df['descendants'].astype(float).astype(int)
hackernews_df['by'] = hackernews_df['by'].astype(str)
hackernews_df['karma'] = hackernews_df['karma'].astype(float).astype(int)
hackernews_df['time'] = pd.to_datetime(hackernews_df['time'], unit='s')
hackernews_df


,id,title,url,score,time,descendants,by,karma
0,1,Y Combinator,http://ycombinator.com,57.0,2006-10-09 18:21:51,15,pg,157234
1,34175557,A Year of Essential Reading on the Current,https://www.criterion.com/current/posts/8024-a...,1.0,2022-12-29 17:53:49,0,cocacola1,3719
2,5402062,"Show HN: Rollbar - reliable, fast, platform-ag...",https://rollbar.com,2.0,2013-03-19 16:57:39,3,brianr,1012
3,21996698,A World Without Pain,https://www.newyorker.com/magazine/2020/01/13/...,2.0,2020-01-08 22:59:56,0,zootme,46
4,35659418,Give roundworms some weed and they’ll get the ...,https://arstechnica.com/science/2023/04/give-a...,1.0,2023-04-21 20:14:44,1,PaulHoule,60158
...,...,...,...,...,...,...,...,...
139996,3675265,"Google Rebrands 'Android Market', It is now 'G...",http://infworm.com/google-rebrands-android-mar...,1.0,2012-03-07 12:47:33,-1,ansrikanth,4
139997,795716,Clenbuterol pills,http://www.fitandshape.com,1.0,2009-08-31 11:09:06,-1,rangilsmith,1
139998,22873968,Save Cost on Cloud,https://medium.com/opsdev/boost-up-infrastruct...,1.0,2020-04-15 02:49:33,0,abhaykumar_,10
139999,9956343,The ionosphere on Venus finally confirmed,http://serious-science.org/colors-of-transit-o...,1.0,2015-07-27 16:48:48,0,mariannadavt,2


In [12]:
hackernews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140001 entries, 0 to 140000
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   id           140001 non-null  int32         
 1   title        140001 non-null  object        
 2   url          140001 non-null  object        
 3   score        140001 non-null  float64       
 4   time         140001 non-null  datetime64[ns]
 5   descendants  140001 non-null  int64         
 6   by           140001 non-null  object        
 7   karma        140001 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(3)
memory usage: 8.0+ MB


In [13]:
hackernews_df.describe()

,id,score,time,descendants,karma
count,1.400010e+05,140001.000000,140001,140001.000000,140001.000000
mean,1.512220e+07,15.381719,2016-10-06 18:12:34.182698752,7.404519,14030.578617
min,1.000000e+00,0.000000,2006-10-09 18:21:51,-1.000000,-54.000000
25%,5.133854e+06,1.000000,2013-01-29 14:49:50,0.000000,141.000000
50%,1.291785e+07,2.000000,2016-11-10 04:54:36,0.000000,1648.000000
75%,2.364891e+07,4.000000,2020-06-26 05:14:15,1.000000,11024.000000
max,3.879956e+07,2873.000000,2023-12-28 22:31:57,1449.000000,384399.000000
std,1.110996e+07,60.624857,NaN,36.510983,30997.233701


Add everything into the feature groups

In [14]:
hackernews_fg = fs.create_feature_group(
    name="hackernews_fg",
    version=2,
    description="Hackernews feature group",
    online_enabled=True,
    statistics_config={"enabled": True, "histograms": True, "correlations": True, "exact_uniqueness": True},
    primary_key=["id"],
    event_time='time',
)
hackernews_fg.insert(hackernews_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/197783/fs/197702/fg/394622


Uploading Dataframe: 0.00% |          | Rows 0/140001 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: hackernews_fg_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/197783/jobs/named/hackernews_fg_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1401b9010>, None)

In [10]:
fs = project.get_feature_store("id2223_enric")
hackernews_fg = fs.get_feature_group("hackernews_fg", 2)
hackernews_fg.show(10)

Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (7.77s) 


,id,title,url,score,time,descendants,by,karma
0,319576,Getting Started with the .NET Task Parallel Li...,http://www.devx.com/dotnet/Article/39204?trk=D...,1.0,2008-09-30 14:14:37+00:00,0,gscott,15038
1,25430706,Safety concerns over planes returning to service,https://www.bbc.com/news/business-55313504,1.0,2020-12-15 14:41:16+00:00,1,CapitalistCartr,27068
2,20756486,The ‘minterns’ taking internships in their 30s,https://www.bbc.com/worklife/article/20190815-...,4.0,2019-08-21 12:01:03+00:00,0,yitchelle,6979
3,27914081,The Cost of Omission: Dr. Valentina Wasson and...,https://chacruna.net/dr-valentina-wasson-and-g...,1.0,2021-07-22 00:50:44+00:00,0,benbreen,40737
4,32009167,A Structural Optimization Tutorial,https://greydanus.github.io/2022/05/08/structu...,2.0,2022-07-07 02:04:08+00:00,0,antiviral,1272
5,24175351,Altucher: NYC Is Dead Forever,https://jamesaltucher.com/blog/nyc-is-dead-for...,65.0,2020-08-16 05:12:06+00:00,41,tomcam,19016
6,18238792,"The strange, sad history of the lobotomy (2011)",https://io9.gizmodo.com/5787430/the-strange-sa...,2.0,2018-10-17 13:39:14+00:00,0,isomorph,4694
7,18804267,Hard Choices – Software Engineering Game Theor...,https://resources.sei.cmu.edu/asset_files/Whit...,2.0,2019-01-02 04:55:00+00:00,0,bribri,1245
8,26631016,"Satellites, space debris may have already brig...",https://www.theregister.com/2021/03/29/satelli...,2.0,2021-03-30 03:32:20+00:00,0,samizdis,33294
9,34988102,"China, Belarus presidents call for establishin...",https://www.aljazeera.com/news/2023/3/1/china-...,3.0,2023-03-01 20:47:55+00:00,1,mikhael,3627
